In [1]:
# Following https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb

In [2]:
!pip install wandb datasets transformers sacrebleu==1.5.1 -qqq

     |████████████████████████████████| 1.7 MB 4.5 MB/s 
     |████████████████████████████████| 270 kB 44.4 MB/s 
     |████████████████████████████████| 2.9 MB 38.0 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 97 kB 6.9 MB/s 
     |████████████████████████████████| 139 kB 48.2 MB/s 
     |████████████████████████████████| 180 kB 53.0 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 1.3 MB 30.8 MB/s 
     |████████████████████████████████| 243 kB 53.0 MB/s 
     |████████████████████████████████| 125 kB 49.1 MB/s 
     |████████████████████████████████| 3.3 MB 36.8 MB/s 
     |████████████████████████████████| 636 kB 36.6 MB/s 
     |████████████████████████████████| 895 kB 48.2 MB/s 
     |████████████████████████████████| 160 kB 46.6 MB/s 
     |████████████████████████████████| 271 kB 42.1 MB/s 


In [3]:
!pip install bert-score -qqq

     |████████████████████████████████| 59 kB 2.2 MB/s 


In [5]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [6]:
!nvidia-smi

Wed Oct 13 09:10:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
import datetime
model_checkpoint = 't5-small'
fp16 = True
todaydate = datetime.datetime.now().strftime("%Y-%m-%d")
print(todaydate)
gdir = f'drive/My Drive/Colab Notebooks/{todaydate}/'
model_name=f'qald9-translator-{todaydate}-{model_checkpoint}' + '-fp16' if fp16 else ''
model_path='./models/'+model_name
ds_path= 'qald-text-to-sparql'

2021-10-13


In [8]:
!unzip qald-text-to-sparql.zip

Archive:  qald-text-to-sparql.zip
 extracting: qald-text-to-sparql/dataset_dict.json  
  inflating: qald-text-to-sparql/test.csv  
   creating: qald-text-to-sparql/test/
  inflating: qald-text-to-sparql/test/dataset.arrow  
  inflating: qald-text-to-sparql/test/dataset_info.json  
  inflating: qald-text-to-sparql/test/state.json  
  inflating: qald-text-to-sparql/train.csv  
   creating: qald-text-to-sparql/train/
  inflating: qald-text-to-sparql/train/dataset.arrow  
  inflating: qald-text-to-sparql/train/dataset_info.json  
  inflating: qald-text-to-sparql/train/state.json  


In [9]:
# Flexible integration for any Python script
import wandb

# 1. Start a W&B run
wandb.init(project='text-to-sparql', entity='shahriar', name= f'{todaydate}-{model_checkpoint}-QALD')

# 2. Save model inputs and hyperparameters
config = wandb.config
config.learning_rate = 0.01

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [10]:
model_name

'qald9-translator-2021-10-13-t5-small-fp16'

In [11]:
!mkdir models

In [12]:
from datasets import load_dataset, load_metric, Dataset, load_from_disk

In [13]:
raw_datasets = load_from_disk(ds_path)

In [ ]:
# raw_datasets = load_dataset("yazdipour/text-to-sparql-t5-lc-quad-v2", data_files={"train": "train.csv", "test": "test.csv"})

In [ ]:
# !pip install tqdm==4.49.0

In [20]:
from tqdm import tqdm

In [ ]:
# Preprocessing

In [17]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [18]:
print(raw_datasets['test']['translation'][0]['sparql'],'\n',
      raw_datasets['test']['translation'][0]['en'],'\n',
      tokenizer(raw_datasets['test']['translation'][0]['en']))

select distinct ?uri where [ res:Salt_Lake_City onto:timeZone ?uri ] 
 what is the time zone of salt lake city? 
 {'input_ids': [125, 19, 8, 97, 2901, 13, 3136, 6957, 690, 58, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [21]:
max_input_length = 0 
max_target_length = 0
for d in tqdm(raw_datasets['train']['translation']):
    len_en = len(d['en'])
    len_qry = len(d['sparql'])
    if len_en > max_input_length: max_input_length=len_en
    if len_qry > max_target_length: max_target_length=len_qry
print('\n',max_input_length, max_target_length)

100%|██████████| 408/408 [00:00<00:00, 994349.82it/s]


 85 299


In [22]:
source_lang = "en"
target_lang = "sparql"
prefix = "translate en to sparql: "

def preprocess_function(examples):
    inputs = []
    targets= []
    for ex in examples["translation"]:
      inputs.append(prefix + ex[source_lang])
      targets.append(ex[target_lang])
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
      labels = tokenizer(targets, max_length=max_target_length, truncation=True)
      
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [23]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 408
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 150
    })
})

In [ ]:
# Fine-tuning the model

In [28]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [29]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=fp16,
)

In [30]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [31]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.replace('?',' ?').replace('.', ' .').strip() for pred in preds]
    labels = [[label.replace('?',' ?').replace('.', ' .').strip()] for label in labels]
    return preds, labels

In [ ]:
# The last thing to define for our Seq2SeqTrainer is how to compute 
# the metrics from the predictions. We need to define 
# a function for this, which will just use the metric we loaded earlier, 
# and we have to do a bit of pre-processing to decode the predictions into texts:

In [32]:
from bert_score import BERTScorer
scorer = BERTScorer(lang="en", rescale_with_baseline=True)
metric = load_metric("sacrebleu")

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

In [36]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    P, R, F1 = scorer.score(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    gen_len = np.mean(prediction_lens)
    
    return {"gen_len":gen_len, 'P':P.mean(), 'R':R.mean(), 'F1':F1.mean(), "bleu-score": result["score"], "bleu-precisions": result["precisions"], "bleu-bp": result["bp"]}

In [40]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [41]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation.
***** Running training *****
  Num examples = 408
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 102
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1357: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,


Epoch,Training Loss,Validation Loss,Gen Len,P,R,F1,Bleu-score,Bleu-precisions,Bleu-bp
1,No log,3.809998,15.793333,"tensor([ 0.1156, 0.0478, -0.1363, 0.0619, -0.0619, -0.0015, -0.0685, 0.0677, 0.0599, -0.1583, 0.0166, 0.0908, -0.0844, -0.1028, 0.1284, 0.1247, 0.0481, -0.0196, 0.0215, -0.1423, -0.1358, 0.1307, 0.0221, 0.0572, -0.0715, -0.1952, -0.1560, -0.3864, 0.1809, -0.0732, 0.0195, -0.3665, -0.0450, -0.0293, -0.0336, 0.1327, -0.3169, -0.1182, -0.2928, -0.0932, -0.0584, 0.0142, -0.2605, -0.0062, -0.2775, -0.0189, -0.0899, -0.0610, 0.0228, -0.0441, -0.4377, 0.0893, -0.4569, -0.0068, -0.2029, 0.0330, 0.1465, 0.0163, 0.0121, -0.1958, 0.0041, -0.0581, 0.0331, 0.0543, -0.3803, -0.3923, 0.0008, 0.0658, -0.0335, 0.2491, -0.0356, 0.0622, -0.0529, -0.0495, -0.0804, 0.0881, -0.1528, 0.0330, -0.0126, 0.0599, -0.1202, -0.1063, 0.0328, 0.0638, -0.0959, -0.0593, -0.2081, -0.5551, -0.0082, -0.1639, -0.3326, 0.0112, -0.3745, -0.1734, -0.0191, -0.1882, -0.0250, 0.0386, 0.0230, -0.0194, -0.0637, 0.0751, -0.0216, 0.0565, 0.0230, -0.0565, 0.1204, -0.0694, 0.1260, -0.4153, -0.0909, -0.2736, 0.0689, 0.1506, -0.5067, -0.0789, 0.0952, 0.0097, -0.2407, -0.0821, -0.0532, -0.3515, 0.0751, 0.1221, -0.4603, -0.3947, 0.0504, -0.0204, 0.0450, -0.0955, -0.0626, -0.5034, 0.0486, 0.1199, -0.4109, 0.1597, -0.0200, -0.0792, 0.0471, -0.0050, -0.3023, 0.0137, -0.3143, 0.1465, -0.2656, 0.0943, 0.0093, 0.0784, -0.4917, -0.2353])","tensor([-0.2637, -0.2946, -0.5433, -0.3912, -0.3972, -0.5682, -0.4814, -0.2743, -0.1824, -0.3942, -0.5119, -0.3619, -0.4845, -0.6629, -0.3746, -0.2197, -0.3294, -0.5080, -0.4468, -0.5816, -0.4520, -0.4759, -0.4552, -0.5454, -0.3053, -0.4533, -0.2942, -0.3337, -0.2650, -0.4640, -0.2354, -0.4735, -0.3893, -0.4695, -0.3393, -0.3530, -0.5231, -0.3746, -0.4583, -0.5653, -0.5303, -0.2778, -0.5848, -0.5292, -0.4269, -0.3042, -0.4201, -0.3280, -0.2556, -0.2893, -0.6235, -0.3416, -0.4556, -0.3771, -0.4137, -0.2353, -0.4610, -0.5053, -0.4472, -0.5010, -0.2794, -0.4755, -0.3565, -0.2567, -0.4280, -0.6268, -0.3725, -0.5272, -0.6732, -0.2729, -0.4704, -0.4112, -0.5781, -0.5477, -0.5425, -0.4792, -0.3277, -0.3561, -0.2115, -0.4090, -0.2800, -0.3912, -0.4088, -0.3619, -0.2965, -0.3434, -0.4317, -0.6106, -0.4261, -0.3824, -0.4793, -0.5535, -0.5887, -0.3535, -0.4972, -0.4155, -0.4898, -0.2915, -0.3111, -0.3408, -0.4290, -0.4141, -0.3855, -0.5343, -0.4528, -0.4275, -0.1712, -0.4066, -0.3302, -0.6064, -0.3345, -0.4107, -0.4274, -0.2669, -0.4513, -0.4221, -0.5416, -0.4406, -0.4701, -0.4093, -0.3804, -0.3594, -0.2590, -0.3313, -0.7349, -0.3929, -0.2961, -0.3941, -0.3746, -0.3357, -0.3780, -0.5630, -0.4069, -0.3309, -0.5085, -0.4077, -0.4345, -0.3000, -0.5226, -0.2940, -0.5722, -0.2826, -0.4175, -0.3932, -0.4524, -0.4334, -0.4582, -0.4645, -0.4956, -0.5632])","tensor([-0.0797, -0.1276, -0.3466, -0.1735, -0.2335, -0.3000, -0.2820, -0.1076, -0.0625, -0.2772, -0.2605, -0.1444, -0.2909, -0.3978, -0.1344, -0.0519, -0.1462, -0.2745, -0.2223, -0.3703, -0.2971, -0.1900, -0.2266, -0.2614, -0.1893, -0.3257, -0.2241, -0.3580, -0.0505, -0.2747, -0.1096, -0.4183, -0.2214, -0.2577, -0.1895, -0.1206, -0.4200, -0.2479, -0.3748, -0.3391, -0.3043, -0.1344, -0.4262, -0.2803, -0.3512, -0.1639, -0.2588, -0.1963, -0.1186, -0.1680, -0.5300, -0.1340, -0.4539, -0.1973, -0.3086, -0.1031, -0.1747, -0.2570, -0.2268, -0.3513, -0.1400, -0.2741, -0.1678, -0.1044, -0.4020, -0.5102, -0.1913, -0.2474, -0.3734, -0.0240, -0.2611, -0.1843, -0.3283, -0.3099, -0.3208, -0.2106, -0.2399, -0.1676, -0.1123, -0.1842, -0.1995, -0.2511, -0.1963, -0.1567, -0.1964, -0.2037, -0.3205, -0.5804, -0.2244, -0.2736, -0.4048, -0.2862, -0.4817, -0.2631, -0.2683, -0.3025, -0.2669, -0.1303, -0.1480, -0.1836, -0.2514, -0.1802, -0.2086, -0.2555, -0.2249, -0.2473, -0.0281, -0.2421, -0.1110, -0.5104, -0.2138, -0.3410, -0.1903, -0.0654, -0.4768, -0.2548, -0.2427, -0.2242, -0.3561, -0.2494, -0.2205, -0.3533, -0.0959, -0.1134, -0.5993, -0.3915, -0.1272, -0.2127, -0.1721, -0.2167, -0.2236, -0.5309, -0.1881, -0.1142, -0.4578, 

The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation.
***** Running Evaluation *****
  Num examples = 150
  Batch size = 4
Trainer is attempting to log a value of "tensor([ 0.1156,  0.0478, -0.1363,  0.0619, -0.0619, -0.0015, -0.0685,  0.0677,
         0.0599, -0.1583,  0.0166,  0.0908, -0.0844, -0.1028,  0.1284,  0.1247,
         0.0481, -0.0196,  0.0215, -0.1423, -0.1358,  0.1307,  0.0221,  0.0572,
        -0.0715, -0.1952, -0.1560, -0.3864,  0.1809, -0.0732,  0.0195, -0.3665,
        -0.0450, -0.0293, -0.0336,  0.1327, -0.3169, -0.1182, -0.2928, -0.0932,
        -0.0584,  0.0142, -0.2605, -0.0062, -0.2775, -0.0189, -0.0899, -0.0610,
         0.0228, -0.0441, -0.4377,  0.0893, -0.4569, -0.0068, -0.2029,  0.0330,
         0.1465,  0.0163,  0.0121, -0.1958,  0.0041, -0.0581,  0.0331,  0.0543,
        -0.3803, -0.3923,  0.0008,  0.0658, -0.0335,  0.2491, -0.0356,  0.0622,
        -0.05

TrainOutput(global_step=102, training_loss=3.6453139361213234, metrics={'train_runtime': 46.0558, 'train_samples_per_second': 8.859, 'train_steps_per_second': 2.215, 'total_flos': 9937048829952.0, 'train_loss': 3.6453139361213234, 'epoch': 1.0})

In [ ]:
trainer.save_model(model_path)

In [ ]:
!ls -l --block-size=M {model_path}

In [ ]:
!mkdir drive/MyDrive/models/{model_name}

In [ ]:
!cp {model_path}/* drive/MyDrive/models/{model_name}/